In [19]:
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

In [20]:
od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')

# remove transit zones
od_data = od_data[:-13, :-13, 8:-4]
duration_data = duration_data[:-13, :-13, 8:-4]
distance_data = distance_data[:-13, :-13, 8:-4]

print(od_data.shape)
print(duration_data.shape)
print(distance_data.shape)

(180, 180, 12)
(180, 180, 12)
(180, 180, 12)


In [21]:
def aggregate_data(data, num_sets, nodes_per_set, aggregation='sum'):
    """
    Aggregate data by summing or averaging over groups of nodes.
    
    Args:
    - data (np.array): The input data of shape (nodes, nodes, time).
    - num_sets (int): The number of sets (or clusters) of nodes.
    - nodes_per_set (int): The number of nodes in each set.
    - aggregation (str): The aggregation method ('sum' or 'average').
    
    Returns:
    - np.array: The aggregated data of shape (num_sets, num_sets, time).
    """
    aggregated_data = np.zeros((num_sets, num_sets, data.shape[2]))
    
    for i in range(num_sets):
        for j in range(num_sets):
            subset = data[i*nodes_per_set:(i+1)*nodes_per_set, j*nodes_per_set:(j+1)*nodes_per_set]
            
            if aggregation == 'sum':
                aggregated_data[i, j] = subset.sum(axis=(0, 1))
            elif aggregation == 'average':
                aggregated_data[i, j] = subset.mean(axis=(0, 1))
    
    return aggregated_data

In [22]:
# Ignore the first 10 origin and destination nodes
# Look at only the next 10 origin and destination nodes

od_data = aggregate_data(od_data, 45, 4, aggregation='sum')
duration_data = aggregate_data(duration_data, 45, 4, aggregation='average')
distance_data = aggregate_data(distance_data, 45, 4, aggregation='average')

print(od_data.shape)
print(duration_data.shape)
print(distance_data.shape)

od_data = od_data[0:5, 5:10, :]
duration_data = duration_data[0:5, 5:10, :]
distance_data = distance_data[0:5, 5:10, :]

sum_across_od = od_data.sum(axis=(0, 1))
print("peak demand:", max(sum_across_od.flatten()))

print("20 of peak demand", 0.2*max(sum_across_od.flatten()))
print("cars per station", 0.2*0.2*max(sum_across_od.flatten())/(od_data.shape[0]))

# od_data = od_data[10:15, 15:20, :]
# duration_data = duration_data[10:15, 15:20, :]
# distance_data = distance_data[10:15, 15:20, :]

# od_data = od_data[10:, 10:, :]
# duration_data = duration_data[10:, 10:, :]
# distance_data = distance_data[10:, 10:, :]

(45, 45, 12)
(45, 45, 12)
(45, 45, 12)
peak demand: 11921.0
20 of peak demand 2384.2000000000003
cars per station 95.36800000000002


In [23]:
# set remaing nans to avg
avg_duration = int(np.nanmean(duration_data))
duration_data[np.isnan(duration_data)] = avg_duration
avg_distance = int(np.nanmean(distance_data))
distance_data[np.isnan(distance_data)] = avg_distance

delta_c = 2 # energy step [kWh] 0.75
time_granularity = 0.25 # in h

# cost_per_mile = 2.75 # in $
# cost_per_hour = 33 # in $ as per https://www.taxi-calculator.com/taxi-rate-san-francisco/271
# cost_per_timestep = cost_per_hour * time_granularity

p_travel = 0.0770 # [$ / mi] https://newsroom.aaa.com/wp-content/uploads/2021/08/2021-YDC-Brochure-Live.pdf 
avg_miles_per_h_driving = 9.6 # in m/h from 190x190x24
operational_cost_per_timestep = avg_miles_per_h_driving * time_granularity * p_travel

beta = 0.6 # in $ according to https://www.bls.gov/regions/west/news-release/averageenergyprices_sanfrancisco.htm#:~:text=San%20Francisco%20area%20households%20paid,per%20therm%20spent%20last%20year -> gives cost for kwh (0.30$/kwh)) + 0.30$ for maintenance 

chevy_bolt_capacity = 65 # in kWh
chevy_bolt_usable_capacity = chevy_bolt_capacity * 0.6 # never go below 20% or above 80% of charge
charger_capacity = 50 # assuming 22.5 kW Chargers
charge_time_per_delta_c = math.ceil((delta_c/charger_capacity)/time_granularity)
charge_levels_per_charge_step = math.floor((charger_capacity*time_granularity)/delta_c)

chevy_bolt_range = 230 # range in mi for mild city trips according to https://media.chevrolet.com/media/us/en/chevrolet/2022-bolt-euv-bolt-ev.detail.html/content/Pages/news/us/en/2021/feb/0214-boltev-bolteuv-specifications.html
chevy_bolt_usable_range = chevy_bolt_range*0.6 # never go below 20% or above 80% of charge and assume 10% less efficient because of range https://cleantechnica.com/2017/10/13/autonomous-cars-shorter-range-due-high-power-consumption-computers/
chevy_bolt_kwh_per_mi = (chevy_bolt_usable_capacity/chevy_bolt_usable_range)/0.7
# print(chevy_bolt_kwh_per_mi)
energy_distance = np.ceil(((distance_data * chevy_bolt_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1 # we should always use energy to satisfy a trip
np.save('energy_distance.npy', energy_distance)
# print(np.sum(energy_distance==1.))
# print(energy_distance.max())
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to h
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)

p_energy = np.ones(int(24/time_granularity))*0.16872 # in $/kWh
p_energy[int(16/time_granularity):int(21/time_granularity)] = 0.38195 # peak prices
p_energy[int(9/time_granularity):int(14/time_granularity)] = 0.14545 # super off peak prices
p_energy *= delta_c # in $/ charge level

p_energy = p_energy[int(8 * (1/time_granularity)):int(20 * (1/time_granularity))]

print(p_energy)

# fleet_size = 116616*0.088*2.5 # got number from Justin Lukes optimization with boundary:283905, without boundary:116616
peak_demand = max(sum_across_od.flatten())
fleet_size = peak_demand * 0.2
number_chargelevels = int(chevy_bolt_usable_capacity/delta_c)
number_spatial_nodes = od_data.shape[0]
charge_locations = np.ones(number_spatial_nodes,dtype=bool).tolist()
cars_per_station_capacity = (np.ceil(np.ones(number_spatial_nodes)*fleet_size*0.2/number_spatial_nodes)).tolist()
print(number_chargelevels)

[0.33744 0.33744 0.33744 0.33744 0.2909  0.2909  0.2909  0.2909  0.2909
 0.2909  0.2909  0.2909  0.2909  0.2909  0.2909  0.2909  0.2909  0.2909
 0.2909  0.2909  0.2909  0.2909  0.2909  0.2909  0.33744 0.33744 0.33744
 0.33744 0.33744 0.33744 0.33744 0.33744 0.7639  0.7639  0.7639  0.7639
 0.7639  0.7639  0.7639  0.7639  0.7639  0.7639  0.7639  0.7639  0.7639
 0.7639  0.7639  0.7639 ]
19


In [24]:
cost_per_mile = 0.91
cost_per_timestep = time_granularity * 0.39

new_tripAttr = []
new_reb_time = []
new_total_acc = []
temp_demand = 0
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time
            temp_demand += attr['demand']
            attr['price'] = cost_per_mile * distance_data[origin,destination,int(ts*time_granularity)] + cost_per_timestep * duration_data[origin,destination,int(ts*time_granularity)] + 2.20 + 2.70 # in $
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)
print(temp_demand)
for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] =  math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spatialNodes'] = number_spatial_nodes
new_data['chargeLevelsPerChargeStep'] = charge_levels_per_charge_step
new_data['episodeLength'] = episode_length
new_data['energy_prices'] = p_energy.tolist()
new_data['chargeLocations'] = charge_locations
new_data['carsPerStationCapacity'] = cars_per_station_capacity
new_data['timeGranularity'] = time_granularity
new_data['operationalCostPerTimestep'] = operational_cost_per_timestep
new_data['peakHours'] = [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

print(episode_length)
print(number_chargelevels)

with open(f'SF_{number_spatial_nodes}.json', 'w') as f:
    json.dump(new_data, f)

95756
48
19
